In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [66]:
fake_data=pd.read_csv('~/Desktop/w207/Final_Project/w207-project/input/fake-and-real-news-dataset/Fake.csv')
true_data=pd.read_csv('~/Desktop/w207/Final_Project/w207-project/input/fake-and-real-news-dataset/True.csv')
fake_data['label']='fake'
true_data['label']='true'
fake_corpus=fake_data[['text','label']]
true_corpus=true_data[['text','label']]
frames=[fake_corpus,true_corpus]
data=pd.concat(frames)

train, test = train_test_split(data, test_size=0.2)

display(train)
display(test)


,text,label
12570,LONDON (Reuters) - Britain said on Monday it w...,true
11232,All of the real evidence of real money and re...,fake
807,While Donald Trump hasn t exactly made an admi...,fake
3556,The Whiner-in-Chief humiliated himself again.T...,fake
21550,Greg Gutfield asks the question we all would l...,fake
...,...,...
15534,TOKYO (Reuters) - U.S. President Donald Trump ...,true
16431,LONDON (Reuters) - Failure to pass the governm...,true
7240,WASHINGTON (Reuters) - Republican President-el...,true
22620,21st Century Wire says The Silicon Valley s te...,fake


,text,label
572,YANGON/NAYPYITAW (Reuters) - Members of the U....,true
7053,A Texas cop is under fire after video surfaced...,fake
13131,BRUSSELS (Reuters) - Prime Minister Theresa Ma...,true
14788,Low energy candidate low energy fans h/t Weas...,fake
852,It s only been a few days since Donald Trump a...,fake
...,...,...
12009,Remember when Pelosi said it s AFFORDABLE :,fake
17866,Late-night TV host Jimmy Kimmel opened his pro...,fake
14087,BEIJING (Reuters) - Some northern Chinese citi...,true
19179,So much for the SCOTUS not being political Che...,fake


In [67]:
from sklearn.feature_extraction import text 
additional_words=['reuters']
stop_words = text.ENGLISH_STOP_WORDS.union(additional_words)
print(stop_words)
print(train['label'].value_counts(),'\n Shape of Data is ',train.shape)
corpus=train['text']
labels=train['label']
test_corpus=test['text']
test_labels=test['label']
cv=CountVectorizer(ngram_range=(1,1),lowercase=True,token_pattern='\w{3,}',stop_words=stop_words)
train_cv=cv.fit_transform(corpus)
test_cv=cv.transform(test_corpus)


frozenset({'an', 'either', 'this', 'forty', 'rather', 'most', 'themselves', 'everything', 'latter', 'interest', 'nor', 'indeed', 'co', 'almost', 'sometimes', 'have', 'twelve', 'whereafter', 'except', 'would', 'ltd', 'fifteen', 'is', 'up', 'of', 'neither', 'your', 'made', 'some', 'thin', 'call', 'though', 'elsewhere', 'himself', 'again', 'eg', 'been', 'not', 'and', 'other', 'there', 'seemed', 'hereby', 'over', 'cant', 'for', 'he', 'even', 'whereupon', 'find', 'somehow', 'any', 'serious', 'moreover', 'empty', 'him', 'we', 'why', 'ourselves', 'still', 'somewhere', 'sometime', 'one', 'where', 'five', 'ever', 'twenty', 'thereupon', 'describe', 'whenever', 'amoungst', 'yet', 'bill', 'nine', 'seems', 'from', 'whole', 'could', 'perhaps', 'first', 'eight', 'she', 'here', 'full', 'ie', 'are', 'yourselves', 'three', 'get', 'nevertheless', 'all', 'may', 'otherwise', 'thereby', 'afterwards', 'can', 'via', 'latterly', 'detail', 'once', 'those', 'much', 'thereafter', 'de', 'a', 'same', 'which', 'belo

In [68]:
log_model=LogisticRegression(solver="liblinear", multi_class="auto")
log_model.fit(train_cv,labels)
pred_labels=log_model.predict(test_cv)
f1_score=round(metrics.f1_score(test_labels,pred_labels,average="weighted"),5)
weights=log_model.coef_
print('Logistic F1_score',f1_score)



Logistic F1_score 0.98452


In [69]:
knn_values=[1,3,5,7,9,11,13]
df_k_stats=pd.DataFrame()
knn_stats={'Model':'KNN','K-value':None,'F1-Score':None} #Create dictionary to hold KNN stats to append with dataframe
for i in knn_values:
    knn_stats['K-value'] = i #Store K-value within dictionary
    knn_model = KNeighborsClassifier(n_neighbors = i) #Create KNN model
    knn_model.fit(train_cv,labels) #Fit model on training data from CountVectorizer fit and transform
    knn_pred=knn_model.predict(test_cv) #Predict on dev dataset
    knn_model_f1=metrics.f1_score(test_labels,knn_pred,average="weighted") #Get f1-score based on the dev labels and predicted values
    knn_stats['F1-Score'] = round(knn_model_f1,4) #Make it pretty
    df_k_stats = df_k_stats.append(knn_stats, ignore_index=True) #place the values in dataframe
print('Show K1 and F1-Score for KNN Model','\n',df_k_stats.to_string(index=False,justify='center'))

Show K1 and F1-Score for KNN Model 
 Model  K-value  F1-Score
 KNN     1.0     0.8353 
 KNN     3.0     0.7884 
 KNN     5.0     0.7715 
 KNN     7.0     0.7635 
 KNN     9.0     0.7576 
 KNN    11.0     0.7533 
 KNN    13.0     0.7463 


In [70]:
df_alpha_stats=pd.DataFrame() #Create dataframe to hold alpha stats
alpha=[.00001,.0001,.001,.01,.1,1,2,5,10] #alpha-values for testing for Naive-Bayes model
for i in alpha:
    nb_stats={'Model':'Naive-Bayes','Alpha':None,'F1-Score':None} #Create dictionary to hold Naive-Bayes stats to append with dataframe
    nb_stats['Alpha']=i #Store alpha-value within dictionary
    nb_model=MultinomialNB(alpha=i) #Create Naive-Bayes model based on alpha value
    nb_model.fit(train_cv,labels) #Fit model on training data from CountVectorizer fit and transform
    nb_pred=nb_model.predict(test_cv) #Predict on dev dataset
    nb_model_f1=metrics.f1_score(test_labels,nb_pred,average="weighted") #Get f1-score based on the dev labels and predicted values
    nb_stats['F1-Score']=round(nb_model_f1,4) #Make it pretty
    df_alpha_stats=df_alpha_stats.append(nb_stats,ignore_index=True)  #place the values in dataframe
print('\nShow Alpha and F1-Score for Naive-Bayes Model','\n',df_alpha_stats.to_string(index=False,justify='center'))



Show Alpha and F1-Score for Naive-Bayes Model 
    Model      Alpha   F1-Score
Naive-Bayes  0.00001   0.9507 
Naive-Bayes  0.00010   0.9503 
Naive-Bayes  0.00100   0.9499 
Naive-Bayes  0.01000   0.9485 
Naive-Bayes  0.10000   0.9462 
Naive-Bayes  1.00000   0.9438 
Naive-Bayes  2.00000   0.9427 
Naive-Bayes  5.00000   0.9402 
Naive-Bayes 10.00000   0.9377 


In [71]:
weights=log_model.coef_ #Get the weights of the logistic regression
sorted_weights=np.argsort(weights)[:,-10:].flatten() #sort them in order to get the top 5 weights per topic
topic_words=cv.get_feature_names_out()[sorted_weights] #Grab the words from the top 5 weights 
categories=['true','fake']
topics=categories#set the topics to place in the dataframe
df_topics=pd.DataFrame() #create dataframe
df_topics['words']=pd.DataFrame(topic_words) #create the column of words for the top 20 words in each category
for i in range(1): #create for loop to go through the topics
    specific_weight=weights[i][sorted_weights] #Get the specific weight for that word
    df_topics[topics[i]]=pd.DataFrame(specific_weight,columns=[topics[i]]) #Place the word and topics in the row
display(df_topics) #showcase the topics and the words for the feature
y_pred_prob = log_model.predict_proba(test_cv)[:, 1]
print(y_pred_prob)


,words,true
0,citing,1.152251
1,barack,1.207690
2,nov,1.432894
3,london,1.502781
4,monday,1.683132
5,tuesday,1.775775
6,friday,1.895099
7,washington,2.038345
8,wednesday,2.068386
9,thursday,2.188648


[1.00000000e+00 8.45314810e-10 9.99940949e-01 ... 9.99991212e-01
 1.35784612e-02 9.99954607e-01]


In [72]:
weights=nb_model.coef_ #Get the weights of the logistic regression
sorted_weights=np.argsort(weights)[:,-10:].flatten() #sort them in order to get the top 5 weights per topic
topic_words=cv.get_feature_names_out()[sorted_weights] #Grab the words from the top 5 weights 
categories=['true','fake']
topics=categories#set the topics to place in the dataframe
df_topics=pd.DataFrame() #create dataframe
df_topics['words']=pd.DataFrame(topic_words) #create the column of words for the top 20 words in each category
for i in range(1): #create for loop to go through the topics
    specific_weight=weights[i][sorted_weights] #Get the specific weight for that word
    df_topics[topics[i]]=pd.DataFrame(specific_weight,columns=[topics[i]]) #Place the word and topics in the row
display(df_topics) #showcase the topics and the words for the feature


/Users/rohin/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


,words,true
0,united,-5.936620
1,republican,-5.882815
2,states,-5.879248
3,house,-5.866152
4,new,-5.840246
5,government,-5.747887
6,state,-5.635886
7,president,-5.339790
8,trump,-4.679511
9,said,-4.088865
